In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import datetime as dt
import time
import statistics

plt.style.use('ggplot')

# 資料匯入與整理

## 台指期資料

In [2]:
df = pd.read_csv('TWF_Futures_Minute_Trade.txt')
df

FileNotFoundError: [Errno 2] No such file or directory: 'TWF_Futures_Minute_Trade.txt'

In [ ]:
df.index = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df = df.drop(columns=['Date','Time'])
df.columns = ['open', 'high', 'low', 'close', 'volume']
df['Hour'] = df.index.map(lambda x: x.hour)
df.head(3)

In [ ]:
rule = '60T'

Morning = df[(df['Hour'] >= 8) & (df['Hour'] <= 13)]
Morning.index = Morning.index + dt.timedelta(minutes=15)

d1 = Morning.resample(rule=rule, closed='right', label='left').first()[['open']]
d2 = Morning.resample(rule=rule, closed='right', label='left').max()[['high']]
d3 = Morning.resample(rule=rule, closed='right', label='left').min()[['low']]
d4 = Morning.resample(rule=rule, closed='right', label='left').last()[['close']]
d5 = Morning.resample(rule=rule, closed='right', label='left').sum()[['volume']]

df_Morning = pd.concat([d1,d2,d3,d4,d5], axis=1)
df_Morning = df_Morning.dropna() #會有沒交易的時段
df_Morning.index = df_Morning.index - dt.timedelta(minutes=15)
df_Morning.head()

In [ ]:
d1 = Morning.resample(rule='1D', closed='right', label='left').first()[['open']]
d2 = Morning.resample(rule='1D', closed='right', label='left').max()[['high']]
d3 = Morning.resample(rule='1D', closed='right', label='left').min()[['low']]
d4 = Morning.resample(rule='1D', closed='right', label='left').last()[['close']]
d5 = Morning.resample(rule='1D', closed='right', label='left').sum()[['volume']]
df_day = pd.concat([d1,d2,d3,d4,d5], axis=1)
df_day

In [ ]:
training_dataset = df_day[(df_day.index >= '2011-01-01') & (df_day.index <= '2018-12-31')]
validation_dataset = df_day[(df_day.index >= '2019-01-01') & (df_day.index <= '2019-12-31')]
test_dataset = df_day[(df_day.index >= '2020-1-1') & (df_day.index <= '2022-5-22')]
in_sample = df_day[(df_day.index >= '2011-01-01') & (df_day.index <= '2019-12-31')]

## 加權指數資料(外部資料)

In [ ]:
PriceMat = pd.read_excel('twa.xlsx', index_col = 0, header = [0])
PriceMat

In [ ]:
PriceMat_trainData = PriceMat[(PriceMat.index >= '2011-01-01') & (PriceMat.index <= '2018-12-31')]
PriceMat_validationData = PriceMat[(PriceMat.index >= '2019-01-01') & (PriceMat.index <= '2019-12-31')]
PriceMat_testData = PriceMat[(PriceMat.index >= '2020-1-1') & (PriceMat.index <= '2022-5-22')]
PriceMat_insample = PriceMat[(PriceMat.index >= '2011-01-01') & (PriceMat.index <= '2019-12-31')]

# 分為訓練集(training)  驗證集(validation)  測試集(testing)

* 訓練集 : 2011-01-01 ~ 2018-12-31 8年
* 驗證集 : 2019-01-01 ~ 2019-12-31 1年
* 測試集 : 2020-01-01 ~ 2022-05-22 約2.5年
* 樣本內 : 2011-01-01 ~ 2019-12-31
* 全樣本 : 2011-01-01 ~ 2022-05-22

In [ ]:
#訓練集
training_dataset  = training_dataset.join(PriceMat_trainData, how='outer')
del training_dataset ['volume']
del training_dataset ['high']
del training_dataset ['low']
training_dataset  = training_dataset.dropna(how='any')
training_dataset 

In [ ]:
#驗證集
validation_dataset  = validation_dataset.join(PriceMat_validationData, how='outer')
del validation_dataset ['volume']
del validation_dataset ['high']
del validation_dataset ['low']
validation_dataset  = validation_dataset .dropna(how='any')
validation_dataset

In [ ]:
#測試集
test_dataset = test_dataset.join(PriceMat_testData, how='outer')
del test_dataset ['volume']
del test_dataset ['high']
del test_dataset ['low']
test_dataset = test_dataset.dropna(how='any')
test_dataset

In [ ]:
#樣本內
in_sample = in_sample.join(PriceMat_insample, how='outer')
del in_sample ['volume']
del in_sample ['high']
del in_sample ['low']
in_sample = in_sample.dropna(how='any')
in_sample

In [ ]:
#全樣本
dataset = df_day.join(PriceMat, how='outer')
del dataset ['volume']
del dataset ['high']
del dataset ['low']
dataset = dataset.dropna(how='any')
dataset 

# 取得結算日資料

* 資料來源：https://www.taifex.com.tw/cht/5/futIndxFSP
* 資料時間：2009/08/19 - 2022/08/03
* 資料調整：
* 每個月第三個禮拜三為結算日
* 當日提前15分鐘交易
* 結算當日進行平倉

In [ ]:
settlementDate_ = pd.read_csv('settlementDate.csv')
settlementDate_.columns = ['settlementDate', 'futures', 'settlementPrice']
bool_ = [False if 'W' in i else True for i in settlementDate_['futures']]
settlementDate = [i.replace('/','-') for i in list(settlementDate_[bool_]['settlementDate'])]
settlementDate = [pd.to_datetime(i).date() for i in settlementDate]
settlementDate

In [ ]:
#如果是結算日，回報為True
bool_ = [False if 'W' in i else True for i in settlementDate_['futures']]

In [ ]:
# 結算日天數：12年*12個月[144天] + 2009.08-2009.12[5天] + 2022[7天]
len(list(settlementDate_[bool_]['futures']))

# 回測架構

* 換月轉倉 : 結算日當天直接平倉
* 進場限制：結算日當天不進場
* 報酬計算：50 * 點數
* 手續費+滑價：單邊 150元

## 樣本內

In [ ]:
train_years = 8
validation_years = 1
test_years = 2 + 142/250
years = train_years + validation_years + test_years

### 訓練集

In [ ]:
df = test_dataset
df

### 最佳化參數:deviate_point、stopwin	

In [ ]:
train_equity = pd.DataFrame()
optimizationList = []

for deviate_point in range(0, 60, 10):
    for stopwin in range(10, 60, 10):
        
        print('----------')
        print(f'deviate_point: {deviate_point}')
        print(f'stopwin: {stopwin / 100}')

        fund = 1000000
        feePaid = 150 #小台指
        length = 15 #設定均線參數
        tax = 0.00002
        BS = None
        profit = []
        unrealize_profit = []
        stoploss = 0.1
        stopwin = stopwin / 100
        date = list(df.index) #抓出所有日期
        trade_times = 0
        buy = []
        sell = []
        sellshort = []
        buytocover = []

        for i in range(len(df)):
            
            budget = (fund + sum(profit)) * 0.1
            
            if i == len(df)-1: #最後一天 

                if BS == None: #若沒部位 終止迴圈

                    profit.append(0)
                    unrealize_profit.append(0)

                elif BS == 'B': #若多頭 平倉

                    if i == t:

                        unrealize_income = tempSize * 50 * (df['close'][i] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                        unrealize_profit.append(unrealize_income)

                    else:

                        unrealize_income = tempSize * 50 * (df['close'][i] - df['close'][i-1])
                        unrealize_profit.append(unrealize_income)

                    income =  tempSize * 50 * (df['close'][i] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                    profit.append(income)
                    sell.append(i)

                elif BS == 'S': #若空頭 平倉   

                    if i == t:

                        unrealize_income = tempSize * 50 * (df['open'][t] - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                        unrealize_profit.append(unrealize_income)

                    else:

                        unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                        unrealize_profit.append(unrealize_income)

                    income = tempSize * 50 * (tempopen - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                    profit.append(income)
                    buytocover.append(i)

                break

            if date[i] in settlementDate:

                if BS == None: #若沒部位 不動作

                    profit.append(0)
                    unrealize_profit.append(0)

                elif BS == 'B': #若多頭 平倉

                    if i == t:

                        unrealize_income = tempSize * 50 * (df['close'][i] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                        unrealize_profit.append(unrealize_income)

                    else:

                        unrealize_income = tempSize * 50 * (df['close'][i] - df['close'][i-1])
                        unrealize_profit.append(unrealize_income)

                    income =  tempSize * 50 * (df['close'][i] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                    profit.append(income)
                    BS = None
                    sell.append(i)

                elif BS == 'S': #若空頭 平倉   

                    if i == t:

                        unrealize_income = tempSize * 50 * (df['open'][t] - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                        unrealize_profit.append(unrealize_income)

                    else:

                        unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                        unrealize_profit.append(unrealize_income)

                    income = tempSize * 50 * (tempopen - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                    profit.append(income)
                    BS = None
                    buytocover.append(i)

            elif BS == None:

                if df['close'][i] + deviate_point < df['twa_close'][i]:

                    trade_times += 1
                    tempSize = budget / 46000
                    tempopen = df['open'][i+1]
                    tempcost = tempSize*46000
                    profit.append(0)
                    unrealize_profit.append(0)
                    BS = 'B' 
                    t = i + 1 #做多時間點
                    buy.append(t)
                    
                elif df['close'][i] > df['twa_close'][i] + deviate_point:

                    trade_times += 1
                    tempSize = budget / 46000
                    tempopen = df['open'][i+1]
                    tempcost = tempSize*46000
                    profit.append(0)
                    unrealize_profit.append(0)
                    BS = 'S'
                    t = i + 1 #放空時間點
                    sellshort.append(t)
                    
                else: 

                    profit.append(0)
                    unrealize_profit.append(0)

            elif BS == 'B':

                if tempSize * 50 * (df['close'][i] - tempopen) * (-1) >= budget * stoploss : #停損

                    if i == t:

                        unrealize_income = tempSize * 50 * (df['open'][i+1] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                        unrealize_profit.append(unrealize_income)

                    else:

                        unrealize_income = tempSize * 50 * (df['open'][i+1] - df['close'][i-1])
                        unrealize_profit.append(unrealize_income)

                    income = tempSize * 50 * (df['open'][i+1] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                    profit.append(income)
                    BS = None
                    sell.append(i+1)

                elif tempSize * 50 * (df['close'][i] - tempopen) >= budget * stopwin : #停利

                    if i == t:
                        unrealize_income = tempSize * 50 * (df['open'][i+1] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax) 
                        unrealize_profit.append(unrealize_income)

                    else:
                        unrealize_income = tempSize * 50 * (df['open'][i+1] - df['close'][i-1]) 
                        unrealize_profit.append(unrealize_income)

                    income = tempSize * 50 * (df['open'][i+1] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                    profit.append(income)
                    BS=None
                    sell.append(i+1)

                else: #只算未實現

                    profit.append(0)
  
                    if i == t:
                        unrealize_income = tempSize * 50 * (df['close'][i] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                        unrealize_profit.append(unrealize_income)

                    else:
                        unrealize_income = tempSize * 50 * (df['close'][i] - df['close'][i-1]) 
                        unrealize_profit.append(unrealize_income)

            elif BS == 'S': 

                if tempSize * 50 * (tempopen - df['close'][i]) * (-1) >= budget * stoploss : #停損

                    if i == t:
                        unrealize_income = tempSize * 50 * (df['open'][t] - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                        unrealize_profit.append(unrealize_income)

                    else:
                        unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                        unrealize_profit.append(unrealize_income)

                    income = tempSize * 50 * (tempopen - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                    profit.append(income)
                    BS = None
                    buytocover.append(i+1)

                elif tempSize * 50 * (tempopen - df['close'][i]) >= budget * stopwin : #停利

                    if i == t:
                        unrealize_income = tempSize * 50 * (df['open'][t] - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                        unrealize_profit.append(unrealize_income)

                    else:
                        unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                        unrealize_profit.append(unrealize_income)

                    income = tempSize * 50 * (tempopen - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                    profit.append(income)
                    BS = None

                else: #只算未實現

                    profit.append(0)

                    if i == t:
                        unrealize_income = tempSize * 50 * (df['open'][t] - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                        unrealize_profit.append(unrealize_income)

                    else:
                        unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i]) 
                        unrealize_profit.append(unrealize_income)

        equity = pd.DataFrame({'profit':np.cumsum(profit), 'unrealize_profit':np.cumsum(unrealize_profit)}, index=df.index)
        equity.plot(grid=True, figsize=(12, 6));
        equity['equity'] = equity['unrealize_profit'] + fund
        equity['drawdown_percent'] = (equity['equity'] / equity['equity'].cummax()) - 1
        equity['drawdown'] = equity['equity'] - equity['equity'].cummax()
        ret = equity['equity'][-1]/equity['equity'][0] - 1
        mdd = abs(equity['drawdown_percent'].min())
        calmarRatio = ret / mdd

        optimizationList.append([deviate_point, stopwin, ret, calmarRatio])


In [ ]:
optResult = pd.DataFrame(optimizationList, columns=['deviate_point', 'stopwin', 'ret', 'calmarRatio'])
optResult.sort_values('ret', ascending=False).iloc[:10] #照ret排序 把前10抓出來

In [ ]:
pic = optResult.pivot('deviate_point', 'stopwin', 'ret')
sns.heatmap(data = pic).set(title='Return');

In [ ]:
pic = optResult.pivot('deviate_point', 'stopwin', 'calmarRatio')
sns.heatmap(data = pic).set(title='Calmar Ratio');

### 驗證集

In [ ]:
df = validation_dataset
df

### 使用訓練集中ret前五的參數得出各項指標

In [ ]:
optimizationList = []
deviate_point = [50, 50, 50, 30, 30]
stopwin = [0.5, 0.4, 0.3, 0.4, 0.2]

for deviate_point, stopwin in zip(deviate_point, stopwin):
    print('----------')
    print(f'deviate_point: {deviate_point}')
    print(f'stopwin: {stopwin}')

    fund = 1000000
    feePaid = 150 #小台指
    length = 15 #設定均線參數
    tax = 0.00002
    BS = None
    #budget = 100000
    profit = []
    unrealize_profit = []
    cost = [] #在判斷交易時 先計入成本 會領先一期
    stoploss = 0.1
    date = list(df.index) #抓出所有日期
    trade_times = 0
    
    for i in range(len(df)):
        
        budget = (fund + sum(profit)) * 0.1
        
        if i == len(df)-1: #最後一天 

            if BS == None: #若沒部位 終止迴圈

                profit.append(0)
                unrealize_profit.append(0)
                cost.append(0)

            elif BS == 'B': #若多頭 平倉

                if i == t:

                    unrealize_income = tempSize * 50 * (df['close'][i] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                    unrealize_profit.append(unrealize_income)

                else:

                    unrealize_income = tempSize * 50 * (df['close'][i] - df['close'][i-1])
                    unrealize_profit.append(unrealize_income)

                income =  tempSize * 50 * (df['close'][i] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                profit.append(income)
                cost.append(0)
                sell.append(i)

            elif BS == 'S': #若空頭 平倉   

                if i == t:

                    unrealize_income = tempSize * 50 * (df['open'][t] - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                    unrealize_profit.append(unrealize_income)

                else:

                    unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                    unrealize_profit.append(unrealize_income)

                income = tempSize * 50 * (tempopen - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                profit.append(income)
                cost.append(0)
                buytocover.append(i)

            break

        if date[i] in settlementDate:

            if BS == None: #若沒部位 不動作

                profit.append(0)
                unrealize_profit.append(0)
                cost.append(0)

            elif BS == 'B': #若多頭 平倉

                if i == t:

                    unrealize_income = tempSize * 50 * (df['close'][i] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                    unrealize_profit.append(unrealize_income)

                else:

                    unrealize_income = tempSize * 50 * (df['close'][i] - df['close'][i-1])
                    unrealize_profit.append(unrealize_income)

                income =  tempSize * 50 * (df['close'][i] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                profit.append(income)
                cost.append(0)
                BS = None
                sell.append(i)

            elif BS == 'S': #若空頭 平倉   

                if i == t:

                    unrealize_income = tempSize * 50 * (df['open'][t] - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                    unrealize_profit.append(unrealize_income)

                else:

                    unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                    unrealize_profit.append(unrealize_income)

                income = tempSize * 50 * (tempopen - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                profit.append(income)
                cost.append(0)
                BS = None
                buytocover.append(i)

        elif BS == None:

            if df['close'][i] + deviate_point < df['twa_close'][i]:

                trade_times += 1
                tempSize = budget / 46000
                tempopen = df['open'][i+1]
                tempcost = tempSize*46000
                profit.append(0)
                unrealize_profit.append(0)
                cost.append(budget)
                BS = 'B' 
                t = i + 1 #做多時間點
                buy.append(t)
                
            elif df['close'][i] > df['twa_close'][i] + deviate_point:

                trade_times += 1
                tempSize = budget / 46000
                tempopen = df['open'][i+1]
                tempcost = tempSize*46000
                profit.append(0)
                unrealize_profit.append(0)
                cost.append(budget)
                BS = 'S'
                t = i + 1 #放空時間點
                sellshort.append(t)
                
            else: 

                profit.append(0)
                unrealize_profit.append(0)
                cost.append(0)

        elif BS == 'B':

            if tempSize * 50 * (df['close'][i] - tempopen) * (-1) >= budget * stoploss : #停損

                if i == t:

                    unrealize_income = tempSize * 50 * (df['open'][i+1] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                    unrealize_profit.append(unrealize_income)

                else:

                    unrealize_income = tempSize * 50 * (df['open'][i+1] - df['close'][i-1])
                    unrealize_profit.append(unrealize_income)

                income = tempSize * 50 * (df['open'][i+1] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                profit.append(income)
                cost.append(0)
                BS = None
                sell.append(i+1)

            elif tempSize * 50 * (df['close'][i] - tempopen) >= budget * stopwin : #停利

                if i == t:
                    unrealize_income = tempSize * 50 * (df['open'][i+1] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax) 
                    unrealize_profit.append(unrealize_income)

                else:
                    unrealize_income = tempSize * 50 * (df['open'][i+1] - df['close'][i-1]) 
                    unrealize_profit.append(unrealize_income)

                income = tempSize * 50 * (df['open'][i+1] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                cost.append(0)
                profit.append(income)
                BS=None
                sell.append(i+1)

            else: #只算未實現

                profit.append(0)
                cost.append(0)

                if i == t:
                    unrealize_income = tempSize * 50 * (df['close'][i] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                    unrealize_profit.append(unrealize_income)

                else:
                    unrealize_income = tempSize * 50 * (df['close'][i] - df['close'][i-1]) 
                    unrealize_profit.append(unrealize_income)

        elif BS == 'S': 

            if tempSize * 50 * (tempopen - df['close'][i]) * (-1) >= budget * stoploss : #停損

                if i == t:
                    unrealize_income = tempSize * 50 * (df['open'][t] - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                    unrealize_profit.append(unrealize_income)

                else:
                    unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                    unrealize_profit.append(unrealize_income)

                income = tempSize * 50 * (tempopen - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                profit.append(income)
                cost.append(0)
                BS = None
                buytocover.append(i+1)

            elif tempSize * 50 * (tempopen - df['close'][i]) >= budget * stopwin : #停利

                if i == t:
                    unrealize_income = tempSize * 50 * (df['open'][t] - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                    unrealize_profit.append(unrealize_income)

                else:
                    unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                    unrealize_profit.append(unrealize_income)

                income = tempSize * 50 * (tempopen - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                profit.append(income)
                cost.append(0)
                BS = None
                buytocover.append(i+1)
                
            else: #只算未實現

                profit.append(0)
                cost.append(0)

                if i == t:
                    unrealize_income = tempSize * 50 * (df['open'][t] - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                    unrealize_profit.append(unrealize_income)

                else:
                    unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i]) 
                    unrealize_profit.append(unrealize_income)  
                    
    equity = pd.DataFrame({'profit':np.cumsum(profit), 'unrealize_profit':np.cumsum(unrealize_profit)}, index=df.index)
    equity.plot(grid=True, figsize=(12, 6));
    equity['equity'] = equity['unrealize_profit'] + fund
    equity['drawdown_percent'] = (equity['equity'] / equity['equity'].cummax()) - 1
    equity['drawdown'] = equity['equity'] - equity['equity'].cummax()
    ret = equity['equity'][-1]/equity['equity'][0] - 1 # 累績報酬
    mdd = abs(equity['drawdown_percent'].min())
    calmarRatio = ret / mdd
    # 年化報酬(%)
    arr = (equity.loc[:,'profit'][-1] / train_years) / fund 
    # 年化波動度
    profit_rate =[]
    for i in range(len(profit)):
        profit_rate.append(profit[i] / fund)
    annual_volatility = statistics.pstdev(profit_rate) * (365 ** 0.5)
    # 年化夏普
    annual_sharpe = arr / annual_volatility
    # 風險報酬比
    risk_reward_ratio = ret / mdd
    # 勝率
    win_times = 0 
    lose_times = 0
    for i in range(len(profit)):
        if profit[i] > 0:
            win_times += 1
        if profit[i] < 0:
            lose_times += 1        
    winning_persent = (trade_times - lose_times) / trade_times 
    optimizationList.append([deviate_point, stopwin, ret, calmarRatio, arr, annual_volatility, mdd, annual_sharpe, risk_reward_ratio, winning_persent])
#     optimizationList.append([deviate_point, stopwin, ret, calmarRatio])

In [ ]:
optResult = pd.DataFrame(optimizationList, columns=['deviate_point', 'stopwin', 'ret', 'calmarRatio', 'arr', 'annual_volatility', 'mdd', 'annual_sharpe', 'risk_reward_ratio', 'winning_persent'])
optResult.sort_values('ret', ascending=False).iloc[:10] #照ret排序 把前10抓出來

## 樣本外

### 測試集

In [ ]:
df = test_dataset
df

### 使用deviate_point = 30 stopwin = 0.4作為策略

In [ ]:
optimizationList = []
buy = []
sell = []
sellshort = []
buytocover = []
fund = 1000000
feePaid = 150 #小台指
tax = 0.00002
BS = None
profit = []
unrealize_profit = []
stoploss = 0.1
stopwin = 0.4
deviate_point = 30
date = list(df.index) #抓出所有日期
trade_times = 0


for i in range(len(df)):
    
    budget = (fund + sum(profit)) * 0.1
    
    if i == len(df)-1: #最後一天 

        if BS == None: #若沒部位 終止迴圈
            
            profit.append(0)
            unrealize_profit.append(0)
        
        elif BS == 'B': #若多頭 平倉
            
            if i == t:
                
                unrealize_income = tempSize * 50 * (df['close'][i] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                
                unrealize_income = tempSize * 50 * (df['close'][i] - df['close'][i-1])
                unrealize_profit.append(unrealize_income)
                
            income =  tempSize * 50 * (df['close'][i] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            sell.append(i)
            
        elif BS == 'S': #若空頭 平倉   
            
            if i == t:
                
                unrealize_income = tempSize * 50 * (df['open'][t] - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                
                unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                unrealize_profit.append(unrealize_income)
                
            income = tempSize * 50 * (tempopen - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            buytocover.append(i)
            
        break

    if date[i] in settlementDate:
       
        if BS == None: #若沒部位 不動作
            
            profit.append(0)
            unrealize_profit.append(0)
        
        elif BS == 'B': #若多頭 平倉
            
            if i == t:
                
                unrealize_income = tempSize * 50 * (df['close'][i] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                
                unrealize_income = tempSize * 50 * (df['close'][i] - df['close'][i-1])
                unrealize_profit.append(unrealize_income)
                
            income =  tempSize * 50 * (df['close'][i] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS = None
            sell.append(i)
            
        elif BS == 'S': #若空頭 平倉   
            
            if i == t:
                
                unrealize_income = tempSize * 50 * (df['open'][t] - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                
                unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                unrealize_profit.append(unrealize_income)
                
            income = tempSize * 50 * (tempopen - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS = None
            buytocover.append(i)
            
    elif BS == None:
        
        if df['close'][i] + deviate_point < df['twa_close'][i]:
            
            trade_times += 1
            tempSize = budget / 46000
            tempopen = df['open'][i+1]
            tempcost = tempSize*46000
            profit.append(0)
            unrealize_profit.append(0)
            BS = 'B' 
            t = i + 1 #做多時間點
            buy.append(t)
            
            
        elif df['close'][i] > df['twa_close'][i] + deviate_point:
            
            trade_times += 1
            tempSize = budget / 46000
            tempopen = df['open'][i+1]
            tempcost = tempSize*46000
            profit.append(0)
            unrealize_profit.append(0)
            BS = 'S'
            t = i + 1 #放空時間點
            sellshort.append(t)
            
            
        else: 
            
            profit.append(0)
            unrealize_profit.append(0)

    elif BS == 'B':
            
        if tempSize * 50 * (df['close'][i] - tempopen) * (-1) >= budget * stoploss : #停損
            
            if i == t:
                
                unrealize_income = tempSize * 50 * (df['open'][i+1] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                
                unrealize_income = tempSize * 50 * (df['open'][i+1] - df['close'][i-1])
                unrealize_profit.append(unrealize_income)
               
            income = tempSize * 50 * (df['open'][i+1] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS = None
            sell.append(i+1)
            
        elif tempSize * 50 * (df['close'][i] - tempopen) >= budget * stopwin : #停利
            
            if i == t:
                unrealize_income = tempSize * 50 * (df['open'][i+1] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax) 
                unrealize_profit.append(unrealize_income)
                
            else:
                unrealize_income = tempSize * 50 * (df['open'][i+1] - df['close'][i-1]) 
                unrealize_profit.append(unrealize_income)
                
            income = tempSize * 50 * (df['open'][i+1] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS=None
            sell.append(i+1)
            
        else: #只算未實現
            
            profit.append(0)
            cost.append(0)
            
            if i == t:
                unrealize_income = tempSize * 50 * (df['close'][i] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                unrealize_income = tempSize * 50 * (df['close'][i] - df['close'][i-1]) 
                unrealize_profit.append(unrealize_income)
                
    elif BS == 'S': 
        
        if tempSize * 50 * (tempopen - df['close'][i]) * (-1) >= budget * stoploss : #停損
            
            if i == t:
                unrealize_income = tempSize * 50 * (df['open'][t] - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                unrealize_profit.append(unrealize_income)
               
            income = tempSize * 50 * (tempopen - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS = None
            buytocover.append(i+1)
           
        elif tempSize * 50 * (tempopen - df['close'][i]) >= budget * stopwin : #停利
            
            if i == t:
                unrealize_income = tempSize * 50 * (df['open'][t] - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                unrealize_profit.append(unrealize_income)
               
            income = tempSize * 50 * (tempopen - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS = None
            buytocover.append(i+1)
            
        else: #只算未實現
            
            profit.append(0)
            
            if i == t:
                unrealize_income = tempSize * 50 * (df['open'][t] - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i]) 
                unrealize_profit.append(unrealize_income)  
                
equity = pd.DataFrame({'profit':np.cumsum(profit), 'unrealize_profit':np.cumsum(unrealize_profit)}, index=df.index)
#equity.plot(grid=True, figsize=(12, 6));
fig, ax = plt.subplots(figsize = (16,6))
ax.ticklabel_format(style='plain')
plt.plot(equity)
equity['equity'] = equity['unrealize_profit'] + fund
equity['drawdown_percent'] = (equity['equity'] / equity['equity'].cummax()) - 1
equity['drawdown'] = equity['equity'] - equity['equity'].cummax()
ret = equity['equity'][-1]/equity['equity'][0] - 1
mdd = abs(equity['drawdown_percent'].min())
calmarRatio = ret / mdd
# 年化報酬(%)
arr = (equity.loc[:,'profit'][-1] / train_years) / fund 
# 年化波動度
profit_rate =[]
for i in range(len(profit)):
    profit_rate.append(profit[i] / fund)
annual_volatility = statistics.pstdev(profit_rate) * (365 ** 0.5)
# 年化夏普
annual_sharpe = arr / annual_volatility
# 風險報酬比
risk_reward_ratio = ret / mdd
# 勝率
win_times = 0 
lose_times = 0
for i in range(len(profit)):
    if profit[i] > 0:
        win_times += 1
    if profit[i] < 0:
        lose_times += 1       
winning_persent = win_times / trade_times 
optimizationList.append([deviate_point, stopwin, ret, calmarRatio, arr, annual_volatility, mdd, annual_sharpe, risk_reward_ratio, winning_persent])

In [ ]:
optResult = pd.DataFrame(optimizationList, columns=['deviate_point', 'stopwin', 'ret', 'calmarRatio','arr', 'annual_volatility', 'mdd', 'annual_sharpe', 'risk_reward_ratio', 'winning_persent'])
optResult

In [ ]:
equity['equity'] = equity['unrealize_profit'] + fund
equity['drawdown_percent'] = (equity['equity'] / equity['equity'].cummax()) - 1
equity['drawdown'] = equity['equity'] - equity['equity'].cummax()
equity

In [ ]:
fig, ax = plt.subplots(figsize = (16,6))

high_index = equity[equity['unrealize_profit'].cummax() == equity['unrealize_profit']].index
equity['unrealize_profit'].plot(label = 'Total Profit', ax = ax, c = 'r', grid=True)
plt.fill_between(equity['drawdown'].index, equity['drawdown'], 0, facecolor  = 'r', label = 'Drawdown', alpha=0.5)
plt.scatter(high_index, equity['unrealize_profit'].loc[high_index],c = '#02ff0f', label = 'High')

plt.legend()
plt.ylabel('Accumulated Profit')
plt.xlabel('Time')
plt.title('Profit & Drawdown',fontsize  = 16);

In [ ]:
fig, ax = plt.subplots(figsize = (16,6))

df['close'].plot(label = 'Close Price', ax = ax, c = 'gray', grid=True, alpha=0.8)
plt.scatter(df['close'].iloc[buy].index, df['close'].iloc[buy],c = 'orangered', label = 'Buy', marker='^', s=60)
plt.scatter(df['close'].iloc[sell].index, df['close'].iloc[sell],c = 'orangered', label = 'Sell', marker='v', s=60)
plt.scatter(df['close'].iloc[sellshort].index, df['close'].iloc[sellshort],c = 'limegreen', label = 'Sellshort', marker='v', s=60)
plt.scatter(df['close'].iloc[buytocover].index, df['close'].iloc[buytocover],c = 'limegreen', label = 'Buytocover', marker='^', s=60)

plt.legend()
plt.ylabel('USD')
plt.xlabel('Time')
plt.title('Price Movement',fontsize  = 16);

## 樣本內的策略表現

In [ ]:
df = in_sample
df

### 使用deviate_point = 30 stopwin = 0.4作為策略

In [ ]:
optimizationList = []
buy = []
sell = []
sellshort = []
buytocover = []
fund = 1000000
feePaid = 150 #小台指 
tax = 0.00002
BS = None
profit = []
unrealize_profit = []
stoploss = 0.1
stopwin = 0.4
deviate_point = 30
date = list(df.index) #抓出所有日期
trade_times = 0


for i in range(len(df)):
    
    budget = (fund + sum(profit)) * 0.1
    
    if i == len(df)-1: #最後一天 

        if BS == None: #若沒部位 終止迴圈
            
            profit.append(0)
            unrealize_profit.append(0)
        
        elif BS == 'B': #若多頭 平倉
            
            if i == t:
                
                unrealize_income = tempSize * 50 * (df['close'][i] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                
                unrealize_income = tempSize * 50 * (df['close'][i] - df['close'][i-1])
                unrealize_profit.append(unrealize_income)
                
            income =  tempSize * 50 * (df['close'][i] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            sell.append(i)
            
        elif BS == 'S': #若空頭 平倉   
            
            if i == t:
                
                unrealize_income = tempSize * 50 * (df['open'][t] - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                
                unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                unrealize_profit.append(unrealize_income)
                
            income = tempSize * 50 * (tempopen - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            buytocover.append(i)
            
        break

    if date[i] in settlementDate:
       
        if BS == None: #若沒部位 不動作
            
            profit.append(0)
            unrealize_profit.append(0)
        
        elif BS == 'B': #若多頭 平倉
            
            if i == t:
                
                unrealize_income = tempSize * 50 * (df['close'][i] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                
                unrealize_income = tempSize * 50 * (df['close'][i] - df['close'][i-1])
                unrealize_profit.append(unrealize_income)
                
            income =  tempSize * 50 * (df['close'][i] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS = None
            sell.append(i)
            
        elif BS == 'S': #若空頭 平倉   
            
            if i == t:
                
                unrealize_income = tempSize * 50 * (df['open'][t] - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                
                unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                unrealize_profit.append(unrealize_income)
                
            income = tempSize * 50 * (tempopen - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS = None
            buytocover.append(i)
            
    elif BS == None:
        
        if df['close'][i] + deviate_point < df['twa_close'][i]:
            
            trade_times += 1
            tempSize = budget / 46000
            tempopen = df['open'][i+1]
            tempcost = tempSize*46000
            profit.append(0)
            unrealize_profit.append(0)
            BS = 'B' 
            t = i + 1 #做多時間點
            buy.append(t)
            
            
        elif df['close'][i] > df['twa_close'][i] + deviate_point:
            
            trade_times += 1
            tempSize = budget / 46000
            tempopen = df['open'][i+1]
            tempcost = tempSize*46000
            profit.append(0)
            unrealize_profit.append(0)
            BS = 'S'
            t = i + 1 #放空時間點
            sellshort.append(t)
            
            
        else: 
            
            profit.append(0)
            unrealize_profit.append(0)

    elif BS == 'B':
            
        if tempSize * 50 * (df['close'][i] - tempopen) * (-1) >= budget * stoploss : #停損
            
            if i == t:
                
                unrealize_income = tempSize * 50 * (df['open'][i+1] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                
                unrealize_income = tempSize * 50 * (df['open'][i+1] - df['close'][i-1])
                unrealize_profit.append(unrealize_income)
               
            income = tempSize * 50 * (df['open'][i+1] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS = None
            sell.append(i+1)
            
        elif tempSize * 50 * (df['close'][i] - tempopen) >= budget * stopwin : #停利
            
            if i == t:
                unrealize_income = tempSize * 50 * (df['open'][i+1] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax) 
                unrealize_profit.append(unrealize_income)
                
            else:
                unrealize_income = tempSize * 50 * (df['open'][i+1] - df['close'][i-1]) 
                unrealize_profit.append(unrealize_income)
                
            income = tempSize * 50 * (df['open'][i+1] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS=None
            sell.append(i+1)
            
        else: #只算未實現
            
            profit.append(0)
            cost.append(0)
            
            if i == t:
                unrealize_income = tempSize * 50 * (df['close'][i] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                unrealize_income = tempSize * 50 * (df['close'][i] - df['close'][i-1]) 
                unrealize_profit.append(unrealize_income)
                
    elif BS == 'S': 
        
        if tempSize * 50 * (tempopen - df['close'][i]) * (-1) >= budget * stoploss : #停損
            
            if i == t:
                unrealize_income = tempSize * 50 * (df['open'][t] - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                unrealize_profit.append(unrealize_income)
               
            income = tempSize * 50 * (tempopen - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS = None
            buytocover.append(i+1)
           
        elif tempSize * 50 * (tempopen - df['close'][i]) >= budget * stopwin : #停利
            
            if i == t:
                unrealize_income = tempSize * 50 * (df['open'][t] - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                unrealize_profit.append(unrealize_income)
               
            income = tempSize * 50 * (tempopen - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS = None
            buytocover.append(i+1)
            
        else: #只算未實現
            
            profit.append(0)
            
            if i == t:
                unrealize_income = tempSize * 50 * (df['open'][t] - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i]) 
                unrealize_profit.append(unrealize_income)  
                
equity = pd.DataFrame({'profit':np.cumsum(profit), 'unrealize_profit':np.cumsum(unrealize_profit)}, index=df.index)
#equity.plot(grid=True, figsize=(12, 6));
fig, ax = plt.subplots(figsize = (16,6))
ax.ticklabel_format(style='plain')
plt.plot(equity)
equity['equity'] = equity['unrealize_profit'] + fund
equity['drawdown_percent'] = (equity['equity'] / equity['equity'].cummax()) - 1
equity['drawdown'] = equity['equity'] - equity['equity'].cummax()
ret = equity['equity'][-1]/equity['equity'][0] - 1
mdd = abs(equity['drawdown_percent'].min())
calmarRatio = ret / mdd
# 年化報酬(%)
arr = (equity.loc[:,'profit'][-1] / train_years) / fund 
# 年化波動度
profit_rate =[]
for i in range(len(profit)):
    profit_rate.append(profit[i] / fund)
annual_volatility = statistics.pstdev(profit_rate) * (365 ** 0.5)
# 年化夏普
annual_sharpe = arr / annual_volatility
# 風險報酬比
risk_reward_ratio = ret / mdd
# 勝率
win_times = 0 
lose_times = 0
for i in range(len(profit)):
    if profit[i] > 0:
        win_times += 1
    if profit[i] < 0:
        lose_times += 1       
winning_persent = win_times / trade_times 
optimizationList.append([deviate_point, stopwin, ret, calmarRatio, arr, annual_volatility, mdd, annual_sharpe, risk_reward_ratio, winning_persent])

In [ ]:
optResult = pd.DataFrame(optimizationList, columns=['deviate_point', 'stopwin', 'ret', 'calmarRatio','arr', 'annual_volatility', 'mdd', 'annual_sharpe', 'risk_reward_ratio', 'winning_persent'])
optResult

In [ ]:
equity['equity'] = equity['unrealize_profit'] + fund
equity['drawdown_percent'] = (equity['equity'] / equity['equity'].cummax()) - 1
equity['drawdown'] = equity['equity'] - equity['equity'].cummax()
equity

### 計算當期報酬、mdd(每年)

In [ ]:
#當期報酬
print('2011ret : ', round((equity['equity']['2012-01-02'] / equity['equity']['2011-01-03']) - 1, 2))
print('2012ret : ', round((equity['equity']['2013-01-02'] / equity['equity']['2012-01-02']) - 1, 2))
print('2013ret : ', round((equity['equity']['2014-01-02'] / equity['equity']['2013-01-02']) - 1, 2))
print('2014ret : ', round((equity['equity']['2015-01-05'] / equity['equity']['2014-01-02']) - 1, 2))
print('2015ret : ', round((equity['equity']['2016-01-04'] / equity['equity']['2015-01-05']) - 1, 2))
print('2016ret : ', round((equity['equity']['2017-01-03'] / equity['equity']['2016-01-04']) - 1, 2))
print('2017ret : ', round((equity['equity']['2018-01-02'] / equity['equity']['2017-01-03']) - 1, 2))
print('2018ret : ', round((equity['equity']['2019-01-02'] / equity['equity']['2018-01-02']) - 1, 2))
print('2019ret : ', round((equity['equity']['2019-12-31'] / equity['equity']['2019-01-02']) - 1, 2))


#mdd
equity_2011 = equity['equity'][( equity.index >= '2011-01-03') & ( equity.index <= '2012-01-02')]
print('mdd_2011 : ', round((equity_2011.max() - equity_2011.min()) / equity_2011.max(), 2))
equity_2012 = equity['equity'][( equity.index >= '2012-01-02') & ( equity.index <= '2013-01-02')]
print('mdd_2012 : ', round((equity_2012.max() - equity_2012.min()) / equity_2012.max(), 2))
equity_2013 = equity['equity'][( equity.index >= '2013-01-02') & ( equity.index <= '2014-01-02')]
print('mdd_2013 : ', round((equity_2013.max() - equity_2013.min()) / equity_2013.max(), 2))
equity_2014 = equity['equity'][( equity.index >= '2014-01-02') & ( equity.index <= '2015-01-05')]
print('mdd_2014 : ', round((equity_2014.max() - equity_2014.min()) / equity_2014.max(), 2))
equity_2015 = equity['equity'][( equity.index >= '2015-01-05') & ( equity.index <= '2016-01-04')]
print('mdd_2015 : ', round((equity_2015.max() - equity_2015.min()) / equity_2015.max(), 2))
equity_2016 = equity['equity'][( equity.index >= '2016-01-04') & ( equity.index <= '2017-01-03')]
print('mdd_2016 : ', round((equity_2016.max() - equity_2016.min()) / equity_2016.max(), 2))
equity_2017 = equity['equity'][( equity.index >= '2017-01-03') & ( equity.index <= '2018-01-02')]
print('mdd_2017 : ', round((equity_2017.max() - equity_2017.min()) / equity_2017.max(), 2))
equity_2018 = equity['equity'][( equity.index >= '2018-01-02') & ( equity.index <= '2019-01-02')]
print('mdd_2018 : ', round((equity_2018.max() - equity_2018.min()) / equity_2018.max(), 2))
equity_2019 = equity['equity'][( equity.index >= '2019-01-02') & ( equity.index <= '2019-12-31')]
print('mdd_2019 : ', round((equity_2019.max() - equity_2019.min()) / equity_2019.max(), 2))

In [ ]:
fig, ax = plt.subplots(figsize = (16,6))

high_index = equity[equity['unrealize_profit'].cummax() == equity['unrealize_profit']].index
equity['unrealize_profit'].plot(label = 'Total Profit', ax = ax, c = 'r', grid=True)
plt.fill_between(equity['drawdown'].index, equity['drawdown'], 0, facecolor  = 'r', label = 'Drawdown', alpha=0.5)
plt.scatter(high_index, equity['unrealize_profit'].loc[high_index],c = '#02ff0f', label = 'High')
plt.legend()
plt.ylabel('Accumulated Profit')
plt.xlabel('Time')
plt.title('Profit & Drawdown',fontsize  = 16);

In [ ]:
fig, ax = plt.subplots(figsize = (16,6))

df['close'].plot(label = 'Close Price', ax = ax, c = 'gray', grid=True, alpha=0.8)
plt.scatter(df['close'].iloc[buy].index, df['close'].iloc[buy],c = 'orangered', label = 'Buy', marker='^', s=60)
plt.scatter(df['close'].iloc[sell].index, df['close'].iloc[sell],c = 'orangered', label = 'Sell', marker='v', s=60)
plt.scatter(df['close'].iloc[sellshort].index, df['close'].iloc[sellshort],c = 'limegreen', label = 'Sellshort', marker='v', s=60)
plt.scatter(df['close'].iloc[buytocover].index, df['close'].iloc[buytocover],c = 'limegreen', label = 'Buytocover', marker='^', s=60)

plt.legend()
plt.ylabel('USD')
plt.xlabel('Time')
plt.title('Price Movement',fontsize  = 16);

## 全樣本的策略表現

In [ ]:
df = dataset
df

In [ ]:
optimizationList = []
buy = []
sell = []
sellshort = []
buytocover = []
fund = 1000000
feePaid = 150 #小台指 
tax = 0.00002
BS = None
profit = []
unrealize_profit = []
stoploss = 0.1
stopwin = 0.4
deviate_point = 30
date = list(df.index) #抓出所有日期
trade_times = 0


for i in range(len(df)):
    
    budget = (fund + sum(profit)) * 0.1
    
    if i == len(df)-1: #最後一天 

        if BS == None: #若沒部位 終止迴圈
            
            profit.append(0)
            unrealize_profit.append(0)
        
        elif BS == 'B': #若多頭 平倉
            
            if i == t:
                
                unrealize_income = tempSize * 50 * (df['close'][i] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                
                unrealize_income = tempSize * 50 * (df['close'][i] - df['close'][i-1])
                unrealize_profit.append(unrealize_income)
                
            income =  tempSize * 50 * (df['close'][i] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            sell.append(i)
            
        elif BS == 'S': #若空頭 平倉   
            
            if i == t:
                
                unrealize_income = tempSize * 50 * (df['open'][t] - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                
                unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                unrealize_profit.append(unrealize_income)
                
            income = tempSize * 50 * (tempopen - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            buytocover.append(i)
            
        break

    if date[i] in settlementDate:
       
        if BS == None: #若沒部位 不動作
            
            profit.append(0)
            unrealize_profit.append(0)
        
        elif BS == 'B': #若多頭 平倉
            
            if i == t:
                
                unrealize_income = tempSize * 50 * (df['close'][i] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                
                unrealize_income = tempSize * 50 * (df['close'][i] - df['close'][i-1])
                unrealize_profit.append(unrealize_income)
                
            income =  tempSize * 50 * (df['close'][i] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS = None
            sell.append(i)
            
        elif BS == 'S': #若空頭 平倉   
            
            if i == t:
                
                unrealize_income = tempSize * 50 * (df['open'][t] - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                
                unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                unrealize_profit.append(unrealize_income)
                
            income = tempSize * 50 * (tempopen - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS = None
            buytocover.append(i)
            
    elif BS == None:
        
        if df['close'][i] + deviate_point < df['twa_close'][i]:
            
            trade_times += 1
            tempSize = budget / 46000
            tempopen = df['open'][i+1]
            tempcost = tempSize*46000
            profit.append(0)
            unrealize_profit.append(0)
            BS = 'B' 
            t = i + 1 #做多時間點
            buy.append(t)
            
            
        elif df['close'][i] > df['twa_close'][i] + deviate_point:
            
            trade_times += 1
            tempSize = budget / 46000
            tempopen = df['open'][i+1]
            tempcost = tempSize*46000
            profit.append(0)
            unrealize_profit.append(0)
            BS = 'S'
            t = i + 1 #放空時間點
            sellshort.append(t)
            
            
        else: 
            
            profit.append(0)
            unrealize_profit.append(0)

    elif BS == 'B':
            
        if tempSize * 50 * (df['close'][i] - tempopen) * (-1) >= budget * stoploss : #停損
            
            if i == t:
                
                unrealize_income = tempSize * 50 * (df['open'][i+1] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                
                unrealize_income = tempSize * 50 * (df['open'][i+1] - df['close'][i-1])
                unrealize_profit.append(unrealize_income)
               
            income = tempSize * 50 * (df['open'][i+1] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS = None
            sell.append(i+1)
            
        elif tempSize * 50 * (df['close'][i] - tempopen) >= budget * stopwin : #停利
            
            if i == t:
                unrealize_income = tempSize * 50 * (df['open'][i+1] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax) 
                unrealize_profit.append(unrealize_income)
                
            else:
                unrealize_income = tempSize * 50 * (df['open'][i+1] - df['close'][i-1]) 
                unrealize_profit.append(unrealize_income)
                
            income = tempSize * 50 * (df['open'][i+1] - tempopen) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS=None
            sell.append(i+1)
            
        else: #只算未實現
            
            profit.append(0)
            cost.append(0)
            
            if i == t:
                unrealize_income = tempSize * 50 * (df['close'][i] - df['open'][t]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                unrealize_income = tempSize * 50 * (df['close'][i] - df['close'][i-1]) 
                unrealize_profit.append(unrealize_income)
                
    elif BS == 'S': 
        
        if tempSize * 50 * (tempopen - df['close'][i]) * (-1) >= budget * stoploss : #停損
            
            if i == t:
                unrealize_income = tempSize * 50 * (df['open'][t] - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                unrealize_profit.append(unrealize_income)
               
            income = tempSize * 50 * (tempopen - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS = None
            buytocover.append(i+1)
           
        elif tempSize * 50 * (tempopen - df['close'][i]) >= budget * stopwin : #停利
            
            if i == t:
                unrealize_income = tempSize * 50 * (df['open'][t] - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i])
                unrealize_profit.append(unrealize_income)
               
            income = tempSize * 50 * (tempopen - df['open'][i+1]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
            profit.append(income)
            BS = None
            buytocover.append(i+1)
            
        else: #只算未實現
            
            profit.append(0)
            
            if i == t:
                unrealize_income = tempSize * 50 * (df['open'][t] - df['close'][i]) - 2 * (tempSize * feePaid + tempSize * 46000 * tax)
                unrealize_profit.append(unrealize_income)
                
            else:
                unrealize_income = tempSize * 50 * (df['close'][i-1] - df['close'][i]) 
                unrealize_profit.append(unrealize_income)  
                
equity = pd.DataFrame({'profit':np.cumsum(profit), 'unrealize_profit':np.cumsum(unrealize_profit)}, index=df.index)
#equity.plot(grid=True, figsize=(12, 6));
fig, ax = plt.subplots(figsize = (16,6))
ax.ticklabel_format(style='plain')
plt.plot(equity)
equity['equity'] = equity['unrealize_profit'] + fund
equity['drawdown_percent'] = (equity['equity'] / equity['equity'].cummax()) - 1
equity['drawdown'] = equity['equity'] - equity['equity'].cummax()
ret = equity['equity'][-1]/equity['equity'][0] - 1
mdd = abs(equity['drawdown_percent'].min())
calmarRatio = ret / mdd
# 年化報酬(%)
arr = (equity.loc[:,'profit'][-1] / train_years) / fund 
# 年化波動度
profit_rate =[]
for i in range(len(profit)):
    profit_rate.append(profit[i] / fund)
annual_volatility = statistics.pstdev(profit_rate) * (365 ** 0.5)
# 年化夏普
annual_sharpe = arr / annual_volatility
# 風險報酬比
risk_reward_ratio = ret / mdd
# 勝率
win_times = 0 
lose_times = 0
for i in range(len(profit)):
    if profit[i] > 0:
        win_times += 1
    if profit[i] < 0:
        lose_times += 1       
winning_persent = win_times / trade_times 
optimizationList.append([deviate_point, stopwin, ret, calmarRatio, arr, annual_volatility, mdd, annual_sharpe, risk_reward_ratio, winning_persent])

In [ ]:
optResult = pd.DataFrame(optimizationList, columns=['deviate_point', 'stopwin', 'ret', 'calmarRatio','arr', 'annual_volatility', 'mdd', 'annual_sharpe', 'risk_reward_ratio', 'winning_persent'])
optResult

In [ ]:
equity['equity'] = equity['unrealize_profit'] + fund
equity['drawdown_percent'] = (equity['equity'] / equity['equity'].cummax()) - 1
equity['drawdown'] = equity['equity'] - equity['equity'].cummax()
equity

### 計算當期報酬、mdd(每年)

In [ ]:
#當期報酬
print('2011ret : ', round((equity['equity']['2012-01-02'] / equity['equity']['2011-01-03']) - 1, 2))
print('2012ret : ', round((equity['equity']['2013-01-02'] / equity['equity']['2012-01-02']) - 1, 2))
print('2013ret : ', round((equity['equity']['2014-01-02'] / equity['equity']['2013-01-02']) - 1, 2))
print('2014ret : ', round((equity['equity']['2015-01-05'] / equity['equity']['2014-01-02']) - 1, 2))
print('2015ret : ', round((equity['equity']['2016-01-04'] / equity['equity']['2015-01-05']) - 1, 2))
print('2016ret : ', round((equity['equity']['2017-01-03'] / equity['equity']['2016-01-04']) - 1, 2))
print('2017ret : ', round((equity['equity']['2018-01-02'] / equity['equity']['2017-01-03']) - 1, 2))
print('2018ret : ', round((equity['equity']['2019-01-02'] / equity['equity']['2018-01-02']) - 1, 2))
print('2019ret : ', round((equity['equity']['2020-01-02'] / equity['equity']['2019-01-02']) - 1, 2))
print('2020ret : ', round((equity['equity']['2021-01-04'] / equity['equity']['2020-01-02']) - 1, 2))
print('2021ret : ', round((equity['equity']['2022-01-03'] / equity['equity']['2021-01-04']) - 1, 2))
print('2022ret : ', round((equity['equity']['2022-05-20'] / equity['equity']['2022-01-03']) - 1, 2))
#mdd
equity_2011 = equity['equity'][( equity.index >= '2011-01-03') & ( equity.index <= '2012-01-02')]
print('mdd_2011 : ', round((equity_2011.max() - equity_2011.min()) / equity_2011.max(), 2))
equity_2012 = equity['equity'][( equity.index >= '2012-01-02') & ( equity.index <= '2013-01-02')]
print('mdd_2012 : ', round((equity_2012.max() - equity_2012.min()) / equity_2012.max(), 2))
equity_2013 = equity['equity'][( equity.index >= '2013-01-02') & ( equity.index <= '2014-01-02')]
print('mdd_2013 : ', round((equity_2013.max() - equity_2013.min()) / equity_2013.max(), 2))
equity_2014 = equity['equity'][( equity.index >= '2014-01-02') & ( equity.index <= '2015-01-05')]
print('mdd_2014 : ', round((equity_2014.max() - equity_2014.min()) / equity_2014.max(), 2))
equity_2015 = equity['equity'][( equity.index >= '2015-01-05') & ( equity.index <= '2016-01-04')]
print('mdd_2015 : ', round((equity_2015.max() - equity_2015.min()) / equity_2015.max(), 2))
equity_2016 = equity['equity'][( equity.index >= '2016-01-04') & ( equity.index <= '2017-01-03')]
print('mdd_2016 : ', round((equity_2016.max() - equity_2016.min()) / equity_2016.max(), 2))
equity_2017 = equity['equity'][( equity.index >= '2017-01-03') & ( equity.index <= '2018-01-02')]
print('mdd_2017 : ', round((equity_2017.max() - equity_2017.min()) / equity_2017.max(), 2))
equity_2018 = equity['equity'][( equity.index >= '2018-01-02') & ( equity.index <= '2019-01-02')]
print('mdd_2018 : ', round((equity_2018.max() - equity_2018.min()) / equity_2018.max(), 2))
equity_2019 = equity['equity'][( equity.index >= '2019-01-02') & ( equity.index <= '2020-01-02')]
print('mdd_2019 : ', round((equity_2019.max() - equity_2019.min()) / equity_2019.max(), 2))
equity_2020 = equity['equity'][( equity.index >= '2020-01-02') & ( equity.index <= '2021-01-04')]
print('mdd_2020 : ', round((equity_2020.max() - equity_2020.min()) / equity_2020.max(), 2))
equity_2021 = equity['equity'][( equity.index >= '2021-01-04') & ( equity.index <= '2022-01-03')]
print('mdd_2021 : ', round((equity_2021.max() - equity_2021.min()) / equity_2021.max(), 2))
equity_2022 = equity['equity'][( equity.index >= '2022-01-03') & ( equity.index <= '2022-05-20')]
print('mdd_2022 : ', round((equity_2022.max() - equity_2022.min()) / equity_2022.max(), 2))

In [ ]:
fig, ax = plt.subplots(figsize = (16,6))

high_index = equity[equity['unrealize_profit'].cummax() == equity['unrealize_profit']].index
equity['unrealize_profit'].plot(label = 'Total Profit', ax = ax, c = 'r', grid=True)
plt.fill_between(equity['drawdown'].index, equity['drawdown'], 0, facecolor  = 'r', label = 'Drawdown', alpha=0.5)
plt.scatter(high_index, equity['unrealize_profit'].loc[high_index],c = '#02ff0f', label = 'High')
plt.vlines('2019-01-01', 0, 2500000, label = 'validation', color = 'gray', alpha=0.5)
plt.vlines('2020-01-01', 0, 2500000, label = 'test', alpha=0.5)
plt.legend()
plt.ylabel('Accumulated Profit')
plt.xlabel('Time')
plt.title('Profit & Drawdown',fontsize  = 16);

In [ ]:
fig, ax = plt.subplots(figsize = (16,6))

df['close'].plot(label = 'Close Price', ax = ax, c = 'gray', grid=True, alpha=0.8)
plt.scatter(df['close'].iloc[buy].index, df['close'].iloc[buy],c = 'orangered', label = 'Buy', marker='^', s=60)
plt.scatter(df['close'].iloc[sell].index, df['close'].iloc[sell],c = 'orangered', label = 'Sell', marker='v', s=60)
plt.scatter(df['close'].iloc[sellshort].index, df['close'].iloc[sellshort],c = 'limegreen', label = 'Sellshort', marker='v', s=60)
plt.scatter(df['close'].iloc[buytocover].index, df['close'].iloc[buytocover],c = 'limegreen', label = 'Buytocover', marker='^', s=60)

plt.legend()
plt.ylabel('USD')
plt.xlabel('Time')
plt.title('Price Movement',fontsize  = 16);